In [ ]:
pip install pytorch-adapt

In [ ]:
import sys

sys.path.insert(0, "../../src")

### Adapters Initialization

In [ ]:
import torch

from pytorch_adapt.adapters import DANN
from pytorch_adapt.containers import Models

G = torch.nn.Linear(1000, 100)
C = torch.nn.Linear(100, 10)
D = torch.nn.Sequential(torch.nn.Linear(100, 1), torch.nn.Flatten(start_dim=0))
models = Models({"G": G, "C": C, "D": D})

adapter = DANN(models=models)

### Adapters Training Step

In [ ]:
device = torch.device("cuda")
adapter.models.to(device)

data = {
    "src_imgs": torch.randn(32, 1000),
    "target_imgs": torch.randn(32, 1000),
    "src_labels": torch.randint(0, 10, size=(32,)),
    "src_domain": torch.zeros(32),
    "target_domain": torch.zeros(32),
}

loss = adapter.training_step(data, device)

### Adapters Inference

In [ ]:
data = torch.randn(32, 1000).to(device)
features, logits = adapter.inference(data)

### Containers Create With

In [ ]:
import torch

from pytorch_adapt.containers import LRSchedulers, Models, Optimizers

G = torch.nn.Linear(1000, 100)
C = torch.nn.Linear(100, 10)
D = torch.nn.Linear(100, 1)

models = Models({"G": G, "C": C, "D": D})
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.456}))
schedulers = LRSchedulers((torch.optim.lr_scheduler.ExponentialLR, {"gamma": 0.99}))

optimizers.create_with(models)
schedulers.create_with(optimizers)

# optimizers contains an optimizer for G, C, and D
# schedulers contains an LR scheduler for each optimizer

print(models)
print(optimizers)
print(schedulers)

### Containers Merge

In [ ]:
more_models = Models({"X": torch.nn.Linear(20, 1)})
models.merge(more_models)

optimizers = Optimizers((torch.optim.Adam, {"lr": 0.456}))
special_opt = Optimizers((torch.optim.SGD, {"lr": 1}), keys=["G", "X"])
optimizers.merge(special_opt)
optimizers.create_with(models)

# models contains G, C, D, and X
# optimizers:
# - the Adam optimizer with lr 0.456 for models C and D
# - the SGD optimizer with lr 1 for models G and X

print(models)
print(optimizers)

### Datasets Source and Target Datasets

In [ ]:
from torchvision.datasets import MNIST

from pytorch_adapt.datasets import (
    MNISTM,
    CombinedSourceAndTargetDataset,
    SourceDataset,
    TargetDataset,
)

x = MNIST(root=".", train=True, transform=None)
y = MNISTM(root=".", train=True, transform=None)
# x and y return (data, label) tuples
print(x[0])
print(y[0])

x = SourceDataset(x)
y = TargetDataset(y)
# x and y return dictionaries
print(x[0])
print(y[0])

xy = CombinedSourceAndTargetDataset(x, y)
# xy returns a dictionary
print(xy[0])

### Datasets Getters and DataloaderCreator

In [ ]:
from pytorch_adapt.datasets import DataloaderCreator, get_mnist_mnistm

datasets = get_mnist_mnistm(["mnist"], ["mnistm"], folder=".")
dc = DataloaderCreator(batch_size=128)
dataloaders = dc(**datasets)

# datasets and dataloaders are dictionaries
print(datasets)
print(dataloaders)

### Hooks Computing Features

In [ ]:
from pytorch_adapt.hooks import FeaturesHook

G = torch.nn.Linear(1000, 100)
models = {"G": G}
data = {
    "src_imgs": torch.randn(32, 1000),
    "target_imgs": torch.randn(32, 1000),
}

hook = FeaturesHook()

losses, outputs = hook({}, {**models, **data})
# outputs contains src_imgs_features and target_imgs_features
print(outputs.keys())

losses, outputs = hook({}, {**models, **data, **outputs})
# outputs is empty
print(outputs.keys())

hook = FeaturesHook(detach=True)
losses, outputs = hook({}, {**models, **data, **outputs})
# outputs contains
# src_imgs_features_detached and target_imgs_features_detached
print(outputs.keys())

### Hooks Detached Features